##Librerias

In [ ]:
# Instalacion de librerías para web scraping y manipulación de datos
!pip install pandas selenium webdriver-manager
!pip install app-store-scraper # Para reseñas de la App Store de Apple
!pip install google-play-scraper # Para reseñas de Google Play

# Instalacion de librerías para análisis de sentimiento y NLP
!pip install sentiment-analysis-spanish
!pip install matplotlib seaborn wordcloud
!pip install emoji



In [ ]:
!pip uninstall -y urllib3 requests six transformers torch
!pip install requests==2.32.3 urllib3==2.2.3 six==1.16.0 --quiet
!pip install torch==2.8.0 transformers==4.46.2 tqdm --quiet


In [ ]:
import os, sys
os.kill(os.getpid(), 9)


## Fase 1: Recolección de Datos (Web Scraping)

In [ ]:
# 1. Importar las librerías necesarias
import pandas as pd
from google_play_scraper import reviews
from google_play_scraper import Sort
from urllib.parse import urlparse, parse_qs

# --- CONFIGURACIÓN ---
nombre_archivo_excel = 'ListaApps.xlsx'      # Nombre del archivo Excel
nombre_columna_url = 'url_google_play'        # Columna con las URLs de las apps
nombre_columna_app = 'nombre_de_la_app'       # Columna con los nombres de las apps
limite_reseñas = 5000                         # Número máximo de reseñas a descargar
# --------------------


# 2. Cargar tu archivo de Excel en un DataFrame de pandas
try:
    df_apps = pd.read_excel(nombre_archivo_excel)
    print("✅ ¡Excel cargado correctamente!")
    print(df_apps.head())
except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo '{nombre_archivo_excel}'.")
    exit()


# 3. Función para extraer el ID de la aplicación desde la URL
def extraer_id_de_url(url):
    """Extrae el ID de una app desde su URL de Google Play."""
    if not isinstance(url, str):
        return None
    try:
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        return query_params['id'][0]
    except (KeyError, IndexError):
        print(f"⚠️ Advertencia: No se pudo extraer el ID de la URL: {url}")
        return None


# 4. Crear columna con los IDs
df_apps['app_id'] = df_apps[nombre_columna_url].apply(extraer_id_de_url)
df_apps.dropna(subset=['app_id'], inplace=True)

print("\n📱 DataFrame con IDs extraídos:")
print(df_apps.head())


# 5. Descargar las reseñas más recientes de cada app
print("\n--- Iniciando la descarga de reseñas ---")

for index, fila in df_apps.iterrows():
    app_nombre = fila[nombre_columna_app]
    app_id = fila['app_id']

    print(f"\nDescargando reseñas más recientes para: {app_nombre} (ID: {app_id})")

    try:
        # Usamos 'reviews' en lugar de 'reviews_all'
        resultado, _ = reviews(
            app_id,
            lang='es',               # Idioma español
            country='co',            # País Colombia
            sort=Sort.NEWEST,        # Ordenar por las más recientes
            count=limite_reseñas     # Limitar a 5000 reseñas
        )

        # Convertir a DataFrame
        df_reviews = pd.DataFrame(resultado)

        # Guardar en CSV
        nombre_archivo_csv = f'reseñas_{app_nombre.replace(" ", "_")}.csv'
        df_reviews.to_csv(nombre_archivo_csv, index=False)
        print(f"✅ Se guardaron {len(df_reviews)} reseñas en '{nombre_archivo_csv}'")

    except Exception as e:
        print(f"❌ Error al descargar reseñas de {app_nombre}: {e}")

print("\n--- Proceso de descarga finalizado ---")


## Fase 2: Unificación de reseñas y eliminación de columnas innecesarias

In [ ]:
import glob # Librería para buscar archivos por patrón

print("--- Iniciando la unificación de archivos CSV ---")

# 1. Definir el patrón de búsqueda
patron_archivos = 'reseñas_*.csv'

# 2. Usar 'glob' para encontrar todos los archivos que coincidan
archivos_csv_encontrados = glob.glob(patron_archivos)

if not archivos_csv_encontrados:
    print(f"❌ Error: No se encontró ningún archivo con el patrón '{patron_archivos}'.")
    print("Asegúrate de que el primer script se haya ejecutado y generado los archivos.")
else:
    print(f"✅ Se encontraron {len(archivos_csv_encontrados)} archivos para unificar.")

    # 3. Leer cada archivo CSV y guardarlo en una lista de DataFrames
    lista_de_dataframes = []
    for archivo in archivos_csv_encontrados:
        try:
            df_temp = pd.read_csv(archivo)

            # Creamos la columna con el nombre de la app a partir del nombre del archivo
            # Como el formato es 'reseñas_Nombre_De_La_App.csv'

            # 1. Quitamos el prefijo 'reseñas_' y el sufijo '.csv'
            nombre_base = archivo[len('reseñas_'):-len('.csv')]

            # 2. Revertimos los '_' a espacios
            nombre_app_limpio = nombre_base.replace("_", " ")

            # 3. Creamos la nueva columna
            df_temp['nombre_app'] = nombre_app_limpio
            # ------------------------

            lista_de_dataframes.append(df_temp)
            print(f"  ...cargando {archivo} ({len(df_temp)} filas) -> App: {nombre_app_limpio}")

        except Exception as e:
            print(f"⚠️ Advertencia: No se pudo leer el archivo '{archivo}'. Error: {e}")

    # 4. Combinar (concatenar) todos los DataFrames de la lista en uno solo
    if lista_de_dataframes:
        # pd.concat une todos los dataframes
        # ignore_index=True crea un índice nuevo y limpio para el dataframe final
        df = pd.concat(lista_de_dataframes, ignore_index=True)

        print(f"\n ¡Unificación completa! El DataFrame 'df' tiene {len(df)} filas en total.")

        # 5. Guardar el DataFrame unificado en un archivo Excel
        nombre_excel_final = 'final_google_store.xlsx'
        try:
            # Usamos index=False para no guardar el índice de pandas en el Excel
            df.to_excel(nombre_excel_final, index=False)
            print(f"✅ DataFrame unificado guardado exitosamente en '{nombre_excel_final}'")

            # 6. Mostrar un vistazo del DataFrame final (variable 'df')
            print("\n--- Primeras 5 filas del DataFrame 'df' unificado (con la nueva columna) ---")
            print(df.head())

            # Mostrar también las columnas para verificar 'nombre_app'
            print("\nColumnas del DataFrame final:")
            print(df.columns)

        except Exception as e:
            print(f"❌ Error al guardar el archivo Excel: {e}")
            print("Verifica si tienes los permisos necesarios o si el archivo ya está abierto.")
    else:
        print("❌ Error: No se pudo cargar ningún DataFrame para unificar.")

In [ ]:
#Se borran las columnas que no se usarán

columnas_a_borrar = [
    'userName',
    'reviewId',
    'userImage',
    'thumbsUpCount',
    'reviewCreatedVersion',
    'replyContent',
    'repliedAt',
    'appVersion'
]

df = df.drop(columns=columnas_a_borrar, errors='ignore')


In [ ]:
df.head()

In [ ]:
#Aplicamos formato a la columna 'at' (fecha de la reseña)

# 1. Asegurarse de que la columna 'at' sea del tipo datetime
df['at'] = pd.to_datetime(df['at'])

# 2. Normalizar la fecha (eliminar la hora)
#    .dt.normalize() establece la hora, minutos y segundos a cero.
#    Crearemos una nueva columna para mantener el dato original para backup.
df['fecha'] = df['at'].dt.normalize()

# 3. (Opcional) Si se quiere la fecha como un string en formato D/M/A
#     útil solo para visualización final, no para cálculos.
# df['fecha_texto'] = df['at'].dt.strftime('%d/%m/%Y')

# --- Verificación ---
print("Comparación de la columna original 'at' y la nueva 'fecha':")
print(df[['at', 'fecha']].head())



## Fase 3: Limpieza y Preprocesamiento de Datos

In [ ]:

import re
import emoji
import chardet


# 1 print(df.head())

# 2️ Función para corregir errores de codificación
def corregir_codificacion(texto):
    """Corrige problemas de codificación comunes (UTF-8 mal interpretado como latin-1)."""
    if not isinstance(texto, str):
        return texto

    # 1. Intentar corregir maldecodificación típica (Ã³ → ó)
    try:
        texto_corregido = texto.encode('latin1').decode('utf-8')
    except:
        texto_corregido = texto

    # 2. Detectar y corregir codificación restante si es posible
    try:
        resultado = chardet.detect(texto_corregido.encode())
        encoding_detectado = resultado['encoding']
        if encoding_detectado and encoding_detectado.lower() != 'utf-8':
            texto_corregido = texto_corregido.encode(encoding_detectado).decode('utf-8', errors='ignore')
    except:
        pass

    # 3. Reemplazos manuales de patrones frecuentes
    reemplazos = {
        'Ã¡': 'á', 'Ã©': 'é', 'Ã­': 'í', 'Ã³': 'ó', 'Ãº': 'ú',
        'Ã±': 'ñ', 'Ã': 'Á', 'Ã‰': 'É', 'Ã': 'Í', 'Ã“': 'Ó', 'Ãš': 'Ú', 'Ã‘': 'Ñ',
        'â€': '', 'â€™': "'", 'â€œ': '"', 'â€': '"', 'â€“': '-', 'â€¢': '•',
        'â€˜': "'", 'â€¢': '', 'â€”': '-', 'Â': ''
    }
    for k, v in reemplazos.items():
        texto_corregido = texto_corregido.replace(k, v)

    return texto_corregido


# 3️ Función de limpieza completa
def limpiar_texto(texto):
    if not isinstance(texto, str):
        return ""

    # --- Corrección de codificación ---
    texto = corregir_codificacion(texto)

    # --- Normalización ---
    texto = texto.lower()

    # --- Eliminación de ruido ---
    texto = re.sub(r'http\S+|www\S+', '', texto)     # eliminar links
    texto = re.sub(r'@\w+', '', texto)               # eliminar menciones
    texto = re.sub(r'#\w+', '', texto)               # eliminar hashtags
    texto = emoji.replace_emoji(texto, replace='')   # eliminar emojis

    # --- Mantener solo letras y acentos en español ---
    texto = re.sub(r'[^a-záéíóúñü\s]', '', texto)

    # --- Espacios extra ---
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto


# 4️ Aplicar limpieza sobre la columna 'content'
df['texto_limpio'] = df['content'].apply(limpiar_texto)


# 5️ Vista previa del resultado
print("\n Vista previa del texto limpio:")
print(df[['content', 'texto_limpio']].head(10))


# 6️ Guardar dataset limpio
nombre_salida = "reseñas_limpias.csv"
df.to_csv(nombre_salida, index=False, encoding="utf-8-sig")

print(f"\n Archivo '{nombre_salida}' guardado correctamente.")
print(f"Total de reseñas procesadas: {len(df)}")


In [ ]:
#eliminamos las filas donde el campo 'texto_limpio' está vació

print("--- Eliminando filas con 'texto_limpio' vacío ---")

# Guardar el número de filas antes de la limpieza
filas_antes = len(df)

# 1. Reemplazar cadenas vacías o que solo contienen espacios con 'pd.NA'
# ^ -> inicio de la cadena
# \s* -> cero o más caracteres de espacio en blanco (espacios, tabs, etc.)
# $ -> fin de la cadena
df['texto_limpio'] = df['texto_limpio'].replace(r'^\s*$', pd.NA, regex=True)

# 2. Eliminar todas las filas donde 'texto_limpio' sea ahora nulo (pd.NA o NaN)
# Usamos re-asignación (df = ...) en lugar de inplace=True
df = df.dropna(subset=['texto_limpio'])

# 3. Informar el resultado
filas_despues = len(df)
print(f" Se eliminaron {filas_antes - filas_despues} filas.")
print(f"El DataFrame 'df' ahora tiene {filas_despues} filas.")



In [ ]:
df.head()

## Fase 4: análisis de sentimientos con sentiment_spanish

In [ ]:
from sentiment_analysis_spanish import sentiment_analysis

# Inicializar el analizador
analizador = sentiment_analysis.SentimentAnalysisSpanish()

def analizar_sentimiento(texto):
    if not texto.strip(): # Si el texto está vacío
        return 0.5 # Neutral por defecto
    return analizador.sentiment(texto)

# Aplicar el análisis a la columna de texto limpio
# El resultado es un valor entre 0 (negativo) y 1 (positivo)
df['sentimiento_score_spa'] = df['texto_limpio'].apply(analizar_sentimiento)

# Clasificar el sentimiento
def clasificar_sentimiento(score):
    if score > 0.6:
        return 'Positivo'
    elif score < 0.4:
        return 'Negativo'
    else:
        return 'Neutral'

df['sentimiento_spa'] = df['sentimiento_score_spa'].apply(clasificar_sentimiento)
print(df[['texto_limpio', 'sentimiento_spa']].head())

In [ ]:
df.head()

## Fase 5: evaluar resultados

In [ ]:
#Por defecto el usuario en la reseña puede dar una calificación ´score´ de 4 a 5 - vamos a comparar con los resultados de 'sentimiento_spa':

# Filtrar solo reseñas con score 4 o 5
altas = df[df['score'].isin([4, 5])]

# Contar cuántas de esas fueron clasificadas como "Negativo"
negativas_en_altas = altas[altas['sentimiento_spa'] == 'Negativo']

# Calcular la proporción
proporcion = len(negativas_en_altas) / len(altas) if len(altas) > 0 else 0

print(f"Proporción de reseñas con score 4 o 5 clasificadas como 'Negativas': {proporcion:.2%}")


In [ ]:
# Filtrar solo reseñas con score de 1 - 2
bajas = df[df['score'].isin([1,2])]

# Contar cuántas de esas fueron clasificadas como "Positivos"
positivas_en_bajas = bajas[bajas['sentimiento_spa'] == 'Positivo']

# Calcular la proporción
proporcion = len(positivas_en_bajas) / len(bajas) if len(bajas) > 0 else 0

print(f"Proporción de reseñas con score 1 - 2 clasificadas como 'Postivas': {proporcion:.2%}")


In [ ]:
#eliminar la columna: 'sentimiento_score_spa'

df = df.drop(columns=['sentimiento_score_spa'])
#df.head()

## Fase 6: Análisis de sentimientos con transformers

In [ ]:
from transformers import pipeline
import pandas as pd
from tqdm.notebook import tqdm

# Inicializar el analizador de sentimientos
analizador_transformer = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

# Función de análisis
def analizar_sentimiento_transformer(texto):
    if not isinstance(texto, str) or not texto.strip():
        return "Neutral"
    try:
        resultado = analizador_transformer(texto[:512])[0]
        label = resultado['label']
        if label in ['1 star', '2 stars']:
            return 'Negativo'
        elif label == '3 stars':
            return 'Neutral'
        else:
            return 'Positivo'
    except Exception as e:
        print(f"Error analizando texto: {e}")
        return "Neutral"

# Aplicar al DataFrame df
tqdm.pandas()
df['sentimiento_transformer'] = df['texto_limpio'].progress_apply(analizar_sentimiento_transformer)

df[['texto_limpio', 'score', 'sentimiento_transformer']].head()


In [ ]:
df.head()

In [ ]:
# Exportar el DataFrame a un archivo Excel
df.to_excel("sentimientos_transformer.xlsx", index=False)

from google.colab import files
files.download("sentimientos_transformer.xlsx")

## Fase 7: Evaluar resultados

In [ ]:
# Filtrar solo reseñas con score 4 o 5
altas = df[df['score'].isin([4, 5])]

# Contar cuántas de esas fueron clasificadas como "Negativo"
negativas_en_altas = altas[altas['sentimiento_transformer'] == 'Negativo']

# Calcular la proporción
proporcion = len(negativas_en_altas) / len(altas) if len(altas) > 0 else 0

print(f"Proporción de reseñas con score 4 o 5 clasificadas como 'Negativas': {proporcion:.2%}")

In [ ]:
# Filtrar solo reseñas con score de 1 a 2
bajas = df[df['score'].isin([1,2])]

# Contar cuántas de esas fueron clasificadas como "Positivos"
positivas_en_bajas = bajas[bajas['sentimiento_transformer'] == 'Positivo']

# Calcular la proporción
proporcion = len(positivas_en_bajas) / len(bajas) if len(bajas) > 0 else 0

print(f"Proporción de reseñas con score 1 a 3 clasificadas como 'Positivas': {proporcion:.2%}")

In [ ]:
df.info()

## Fase 8: Medir el nivel de concordancia entre el sentimiento predicho (spanish y transformers) y el score del usuario

In [ ]:
# Simplificar el score
df['score_label'] = df['score'].apply(lambda x: 'Positivo' if x >= 4 else ('Negativo' if x <= 2 else 'Neutral'))

# Comparar con el sentimiento de cada modelo
concordancia_spanish = (df['score_label'] == df['sentimiento_spa']).mean()
concordancia_transformer = (df['score_label'] == df['sentimiento_transformer']).mean()

print(f"Concordancia SentimentAnalysisSpanish: {concordancia_spanish:.2%}")
print(f"Concordancia Transformer: {concordancia_transformer:.2%}")


## Fase 9: Gráfico de falsos negativos y falsos positivos (para comparar errores entre los dos modelos)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 1️ Crear etiqueta de sentimiento según el score del usuario
df['score_label'] = df['score'].apply(
    lambda x: 'Positivo' if x >= 4 else ('Negativo' if x <= 2 else 'Neutral')
)

# 2️ Calcular la concordancia entre el sentimiento y el score
concordancia_spanish = (df['score_label'] == df['sentimiento_spa']).mean()
concordancia_transformer = (df['score_label'] == df['sentimiento_transformer']).mean()

# 3️ Resumen con proporciones
datos = {
    'Modelo': ['SentimentAnalysisSpanish', 'Transformer (BERT)'],
    'Negativos con score 4-5 (%)': [28.44, 9.88],
    'Positivos con score 1-2 (%)': [1.52, 3.38],
    'Concordancia (%)': [concordancia_spanish * 100, concordancia_transformer * 100]
}

tabla_comparativa = pd.DataFrame(datos)

# === GRÁFICA 1: Concordancia general ===
plt.figure(figsize=(8, 5))
bars = plt.bar(tabla_comparativa['Modelo'], tabla_comparativa['Concordancia (%)'], color=['#4c72b0', '#55a868'])
plt.title('Concordancia entre el score del usuario y el sentimiento detectado', fontsize=13)
plt.ylabel('Concordancia (%)')
plt.ylim(0, 100)

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 1, f'{yval:.2f}%', ha='center', fontsize=11)

plt.show()

# === GRÁFICA 2: Falsos negativos y positivos ===
ancho_barra = 0.35
x = np.arange(len(tabla_comparativa['Modelo']))

fig, ax = plt.subplots(figsize=(8, 5))
b1 = ax.bar(x - ancho_barra/2, tabla_comparativa['Negativos con score 4-5 (%)'], ancho_barra, label='Falsos Negativos', color='#d62728')
b2 = ax.bar(x + ancho_barra/2, tabla_comparativa['Positivos con score 1-2 (%)'], ancho_barra, label='Falsos Positivos', color='#1f77b4')

ax.set_ylabel('Proporción (%)')
ax.set_title('Comparación de errores de clasificación entre modelos')
ax.set_xticks(x)
ax.set_xticklabels(tabla_comparativa['Modelo'], rotation=10)
ax.legend()

for b in b1 + b2:
    yval = b.get_height()
    ax.text(b.get_x() + b.get_width()/2, yval + 0.5, f'{yval:.2f}%', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

# === Mostrar tabla resumen ===
print("\n📊 Comparativo general de desempeño entre modelos:")
display(tabla_comparativa)


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# --- Preparación ---
# 1. Definir el orden de las etiquetas
# Para que la matriz (Pos, Neg, Neu) se muestre en un orden lógico
# y sea igual para ambos modelos, permitiendo una comparación justa.
labels_order = ['Positivo', 'Negativo', 'Neutral']

# 2. Extraer los datos (Realidad vs. Predicción)
y_real = df['score_label']
y_pred_spa = df['sentimiento_spa']
y_pred_transformer = df['sentimiento_transformer']


# === GRÁFICA 3: Matriz de Confusión - SentimentAnalysisSpanish ===

print("\n--- 1. Matriz de Confusión (SentimentAnalysisSpanish) ---")

# 1. Calcular la Matriz de Confusión
cm_spa = confusion_matrix(y_real, y_pred_spa, labels=labels_order)

# 2. Graficar el Heatmap
plt.figure(figsize=(7, 5))
sns.heatmap(cm_spa,
            annot=True,     # Mostrar los números en cada celda
            fmt='d',        # Formato de los números (enteros)
            cmap='Blues',   # Paleta de colores
            xticklabels=labels_order,
            yticklabels=labels_order)

plt.title('Matriz de Confusión: SentimentAnalysisSpanish')
plt.ylabel('Realidad (Score Usuario)')
plt.xlabel('Predicción del Modelo')
plt.show()


# === GRÁFICA 4: Matriz de Confusión - Transformer (BERT) ===

print("\n--- 2. Matriz de Confusión (Transformer - BERT) ---")

# 1. Calcular la Matriz de Confusión
cm_transformer = confusion_matrix(y_real, y_pred_transformer, labels=labels_order)

# 2. Graficar el Heatmap
plt.figure(figsize=(7, 5))
sns.heatmap(cm_transformer,
            annot=True,
            fmt='d',
            cmap='Oranges', # Usamos otro color para diferenciarlo
            xticklabels=labels_order,
            yticklabels=labels_order)

plt.title('Matriz de Confusión: Transformer (BERT)')
plt.ylabel('Realidad (Score Usuario)')
plt.xlabel('Predicción del Modelo')
plt.show()


# === (Opcional pero recomendado) Reporte de Clasificación ===
# Esto te da la Precisión, Recall y F1-Score para cada clase

print("\n\n--- Reporte de Clasificación (SentimentAnalysisSpanish) ---")
print(classification_report(y_real, y_pred_spa, labels=labels_order, digits=3))

print("\n--- Reporte de Clasificación (Transformer - BERT) ---")
print(classification_report(y_real, y_pred_transformer, labels=labels_order, digits=3))

## Fase 10: Resultados y análisis del sentimiento de las aplicaciones financiereas

In [ ]:
print("--- Generando gráfico de proporciones de sentimiento ---")

# --- 1. Preparar los datos para ordenar ---
# calcular el porcentaje de 'Negativo' para cada app.

# 1a. Contar los sentimientos por app
df_counts = df.groupby('nombre_app')['sentimiento_transformer'].value_counts()

# 1b. Calcular el total de reseñas por app
df_total = df.groupby('nombre_app')['sentimiento_transformer'].count()

# 1c. Calcular la proporción de cada sentimiento
df_proportions = (df_counts / df_total).unstack(fill_value=0)

# 1d. Obtener el order para el gráfico:
# Ordenamos las apps por 'Negativo' de mayor a menor.
if 'Negativo' in df_proportions.columns:
    order_apps = df_proportions.sort_values(by='Negativo', ascending=False).index.tolist() # Convert to list
else:
    print("Advertencia: No se encontraron reseñas 'Negativo' para ordenar.")
    order_apps = df_proportions.index.tolist() # Convert to list (Order by default)


print("Orden de apps (de más negativas a menos):", order_apps)


# --- 2. Configurar el estilo del gráfico ---
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 7))


# --- 3. Graficar usando pandas plot (Stacked Bar) ---

# Reorder the dataframe based on the order_apps list
df_proportions_ordered = df_proportions.reindex(order_apps)

# Define colors for the sentiments
colors = {'Positivo':'#2ca02c', 'Negativo':'#d62728', 'Neutral':'#ff7f0e'}
# Ensure columns are in a desired order for stacking (e.g., Negativo, Neutral, Positivo)
column_order = ['Negativo', 'Neutral', 'Positivo']
# Filter columns to include only those present in df_proportions_ordered
columns_to_plot = [col for col in column_order if col in df_proportions_ordered.columns]


ax = df_proportions_ordered[columns_to_plot].plot(kind='bar', stacked=True, color=[colors[col] for col in columns_to_plot], ax=plt.gca())


# --- 4. Ajustes y limpieza del gráfico ---
ax.set_title('Proporción de Sentimientos por App Financiera', fontsize=16, weight='bold')
ax.set_ylabel('Proporción de Reseñas (100%)')
ax.set_xlabel('Aplicación')

# Ajusta las etiquetas del eje Y para mostrar porcentajes
ax.set_yticklabels(['{:,.0%}'.format(y) for y in ax.get_yticks()])

# Rota etiquetas del eje X si son muchas apps
plt.xticks(rotation=45, ha='right')

# Mover la leyenda fuera del gráfico
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

plt.tight_layout()
plt.show()

# --- 5. Mostrar la tabla de proporciones ---
print("\n--- Tabla de Proporciones (para referencia) ---")
# Multiplicamos por 100 y redondeamos
print(df_proportions.mul(100).round(2).sort_values(by='Negativo', ascending=False))